In [2]:
# TODO testar a rede com XOR

# TODO revisar o backprop, tentar deixar tudo igual as minhas anotacoes,
# questoes de transposicao etc, se nao der, pq n deu?

# TODO fiz classe abstrata Loss, criar um .py para cada loss especializada?
# tbm uma classe para cada otimizador? 
# como organizar melhor as classes e modulos do projeto?

In [3]:
import numpy as np
from nn import NeuralNetwork
from loss import MSE
from activation import ReLU, Sigmoid
from layer import Layer

from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


layers = [Layer(1, 4, 
          ReLU()), 
          Layer(4, 5, 
          ReLU()),
          Layer(5, 6, 
          ReLU()),
          Layer(6, 7, 
          ReLU()),
          Layer(7, 2, 
          ReLU())]


layers = [Layer(1, 1, 
          Sigmoid())]


# layers = [Layer(784, 32, 
#           ReLU()), 
#           Layer(32, 784, 
#           Sigmoid())]

x = np.random.random((1, 1))
nn_my = NeuralNetwork(layers)
mse = MSE(nn_my)
l2 = mse(x, x)
pred = nn_my(x)
print(nn_my)
mse.backward()

NeuralNetwork(
   Layer 0: (1, 1) 
)


In [4]:
import pandas as pd

data = pd.read_csv("data/Housing.csv")
X_house, Y_house = data.price, data.area
X_house = np.array(X_house)
Y_house = np.array(Y_house)

X_house_normalized = (X_house - np.min(X_house)) / (np.max(X_house) - np.min(X_house))
Y_house_normalized = (Y_house - np.min(Y_house)) / (np.max(Y_house) - np.min(Y_house))

X_house_normalized = X_house_normalized.reshape(-1, 1)
Y_house_normalized = Y_house_normalized.reshape(-1, 1)

print(X_house_normalized.shape)
batch_size = 32
X_house_normalized = X_house_normalized.reshape(-1, 1, batch_size)
Y_house_normalized = Y_house_normalized.reshape(-1, 1, batch_size)

(544, 1)


In [ ]:
import torch.nn as nn

class NNT(nn.Module):


    def __init__(self):
        super(NNT, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 32),
            nn.ReLU(),
            nn.Linear(32, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        activations = []
        for layer in self.model:
            x = layer(x)
            activations.append(x.clone())
        return x, activations

    def forward_original(self, x):
        return self.model(x)

from torch import optim
nnt = NNT()
opt = optim.SGD(nnt.parameters(), lr=0.05)
loss_t = nn.MSELoss()

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST(root='data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [ ]:
xor_data = np.array([[0,0], [0,1], [1,0], [1,1]])
xor_y = np.array([[0, 1, 1, 0]])

In [10]:
from abc import ABC, abstractmethod

class Optimizer(ABC):
    def __init__(self, model: NeuralNetwork, lr: float):
        self.model = model
        self.lr = lr

    @abstractmethod
    def step(self):
        pass

    @abstractmethod
    def zero_grad(self):
        pass

class Adam(Optimizer):
    
    def step(self):
        layers = self.model.get_layers()
        for layer in layers:
            # TODO investigar esses transpostos, tentar deixar o backprop igual as minhas anotacoes?
            layer.W -= self.lr * layer.grad_local.T
            layer.b -= self.lr * np.sum(layer.delta, axis=1)

    def zero_grad(self):
        layers = self.model.get_layers()
        for layer in layers:
            layer.delta = 0
            layer.grad_local = 0

In [11]:
optimizer = Adam(nn_my, lr = 0.005)

In [12]:
EPOCHS = 50000
LOSS_T = []
epoch_loss = []
import torch
for i in range(EPOCHS):
    LOSS = 0
    for x, y in zip(X_house_normalized, Y_house_normalized):
        pred = nn_my(x)
        loss = mse(pred, y)
        # print("x", x)
        # print("w", nn_my.layers[0].W)
        # print("b", nn_my.layers[0].b)
        # print("pred", pred)
        # print("loss" , loss)
        LOSS += loss
        mse.backward()
        optimizer.step()
        optimizer.zero_grad()
    epoch_loss.append(np.mean(LOSS))
    print(i, epoch_loss[i])

0 0.36802740156699765
1 0.36641944425673684
2 0.3650722184951836
3 0.3639286113095393
4 0.36294460395385075
5 0.36208609713105466
6 0.36132655705868233
7 0.3606452556003943
8 0.36002594446572034
9 0.35945584940375686
10 0.35892490225234036
11 0.35842515114910445
12 0.357950305144023
13 0.3574953808711554
14 0.35705642719323344
15 0.3566303097527591
16 0.3562145417880508
17 0.35580715084942766
18 0.3554065734945663
19 0.35501157187692156
20 0.3546211675274059
21 0.35423458868313606
22 0.35385122832226445
23 0.35347061068251245
24 0.353092364518544
25 0.35271620172365414
26 0.3523419002296412
27 0.3519692903242336
28 0.3515982437023806
29 0.3512286647070502
30 0.35086048332522196
31 0.3504936495919263
32 0.3501281291244181
33 0.34976389956367504
34 0.3494009477443899
35 0.3490392674497678
36 0.34867885763559264
37 0.34831972103059694
38 0.34796186303830834
39 0.3476052908801179
40 0.3472500129310605
41 0.3468960382092471
42 0.34654337598751384
43 0.346192035502004
44 0.34584202573735734


In [ ]:
epoch_loss

In [ ]:
pred.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(range(1, len(epoch_loss) + 1), epoch_loss)
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

In [ ]:
isinstance( np.random.randn(1, 1)[0][0], np.float64)

In [ ]:
print(-8.36929747e+10)

In [ ]:
EPOCHS = 10000
LOSS_T = []
LOSS = []
import torch
for i in range(EPOCHS):
    #for data in train_loader:
    #x, _ = data
    #x = x.reshape((784, -1)).numpy()
    pred = nn_my(a)
    loss = mse(pred, a)
    LOSS.append(loss)
    mse.backward()
    optimizer(nn_my)
    #zero_grad(nn_my)
    ## torch
    #x = x.reshape((-1, 784))
    #x = torch.asarray(x)
    #pred_t, activations = nnt(x)
    #loss = loss_t(pred_t, x)
    #LOSS_T.append(loss)
    #loss.backward()
    #opt.step()
    #opt.zero_grad()

In [ ]:
for i in range(5-1, -1, -1):
    print(i)

In [ ]:
import matplotlib.pyplot as plt
import torch

input_image_tensor = a.reshape(1, 1, 28, 28)   # torch.Tensor
torch_image_tensor = pred_t.detach().reshape(1, 1, 28, 28)  # torch.Tensor
mine_image_tensor = pred.reshape(1, 1, 28, 28) # numpy.ndarray


input_image_np = input_image_tensor.squeeze()  # Remove singleton dimensions
torch_image_np = torch_image_tensor.squeeze().numpy()
mine_image_np = mine_image_tensor.squeeze()
diff = input_image_np - torch_image_np

fig, axes = plt.subplots(1, 4, figsize=(12, 4))

axes[0].imshow(input_image_np, cmap='gray')  # Use 'gray' colormap for grayscale images
axes[0].set_title('Input Image')
axes[0].axis('off')

axes[1].imshow(torch_image_np, cmap='gray')
axes[1].set_title('Torch Image')
axes[1].axis('off')

axes[2].imshow(mine_image_np, cmap='gray')
axes[2].set_title('Mine Image')
axes[2].axis('off')

axes[3].imshow(diff, cmap='gray')
axes[3].set_title('Difference')
axes[3].axis('off')
plt.show()

In [ ]:
import numpy as np

random_array = np.random.uniform(low=-1, high=1, size=(3, 4))
print(random_array)

In [ ]:
numpy_array = np.random.randn(3, 3) * np.sqrt(2.0 / (3 + 3))  
numpy_array

In [ ]:
torch.nn.Linear(3,3).weight.data

In [ ]:
torch.nn.Linear(3,3).bias.data

In [ ]:
import matplotlib.pyplot as plt

plt.plot(LOSS, label='Loss')
plt.xlabel('Batch Iterations')
plt.ylabel('Loss')
plt.title('Loss Over Batch Iterations')
plt.legend()
plt.show()

In [ ]:
print(nn)

In [ ]:
for l, layer in enumerate(nn_my.layers):
    print(f'l= {l}')
    print(f'x shape {layer.x.shape}')
    print(f'W shape {layer.W.shape}')
    print(f'W.T @, x.shape {(layer.W.T.shape, layer.x.shape)} = {(layer.W.T @ layer.x).shape}')
    print(f'B shape {layer.b.shape}')
    print(f'a shape {layer.a.shape}')
    print(f'z shape {layer.z.shape}')
    print(f'GRADS')
    print(f'local {layer.grad_local.shape}')
    print(f'delta {layer.delta.shape}\n')
